In [30]:
import praw #Python Reddit API wrapper
import pandas as pd
from praw.models import MoreComments
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk import FreqDist
import datetime as dt
import re # Remove urls
import emoji
import spacy
import en_core_web_sm

In [12]:
# Create a reddit connection with reddit API information
# Create praw.Reddit object with with reddit OAuth creds
# Reddit application creds created at https://www.reddit.com/prefs/apps
reddit = praw.Reddit(
            client_id= PRAWConfig.REDDIT_CLIENT_ID,
            client_secret= PRAWConfig.REDDIT_CLIENT_SECRET,
            user_agent= PRAWConfig.REDDIT_USER_AGENT)

### Submission Object

In [13]:
# Obtain title,'Submission ID' and date of the 20 'hottest' submissions in r/WallStreetbets that mentions '$AMC'
subreddit = reddit.subreddit('WallStreetBets')
ticker = "AMC" 

def get_date(date):
    return dt.datetime.fromtimestamp(date)

for submission in subreddit.hot(limit=20):
    for submission in subreddit.search(ticker, limit=130):
        if submission.domain != "self.wallstreetbets":
            continue
        print(submission.title)
        print('Submission ID =', submission.id, '\n')
        print('date =', get_date(submission.created_utc) , '\n')
    

"AMC, GameStop shares fall to half of their June peaks" - Time to Buy?! 🚀🚀🚀
Submission ID = okdbfa 

date = 2021-07-14 21:47:59 

Daily Option Statistics for AMC, GME, AAPL, SPCE, CLOV, WISH, TLRY, CLNE, PLTR and others
Submission ID = okcehp 

date = 2021-07-14 21:04:06 

Daily Popular Tickers Thread for July 14, 2021 - AMC | CLOV | NOK
Submission ID = ok16jr 

date = 2021-07-14 11:00:59 

Ride the Magic ShortBus TA DD - SPY, AMC, SPCE, and AAPL - 7/16
Submission ID = okcjtr 

date = 2021-07-14 21:11:07 

Why AMC and GME price tanked
Submission ID = okfa8w 

date = 2021-07-14 23:26:48 

Current State of $GME and $AMC - Unbiased Technical Analysis
Submission ID = ojpqu3 

date = 2021-07-13 22:43:35 

Daily Option Statistics for AMC, CLOV, GME, NOK, SPCE, AAPL, BB, TSLA, SOFI, WISH and others.
Submission ID = ojnpu9 

date = 2021-07-13 21:04:43 

Daily Popular Tickers Thread for July 13, 2021 - SPCE | CLOV | AMC
Submission ID = ojcg2b 

date = 2021-07-13 11:01:06 

Daily Option Statisti

Record Stock Sales From Money-Losing Firms Ring the Alarm Bells
Submission ID = o97np8 

date = 2021-06-27 23:57:12 

Daily option statistics for WISH, AMC, BB, GME, CLNE, CLOV, WKHS, CLF, TSLA, and others.
Submission ID = o542tb 

date = 2021-06-21 21:02:45 

$AMC Covered Options Trading
Submission ID = o5tjsh 

date = 2021-06-22 19:18:04 

Is TD broken or do I actually owe $1,000,000?
Submission ID = o8qy98 

date = 2021-06-27 06:35:49 

Daily Popular Tickers Thread for June 21, 2021 - AMC | SPY | BB
Submission ID = o4rqwu 

date = 2021-06-21 11:00:39 

Identifying bots on WSB
Submission ID = o7t2nt 

date = 2021-06-25 19:05:18 

$AMC Reversed nicely after the New Moon for the perfect setup for this week's Super Full MOON - If History Repeats Itself, Big Move Incoming!
Submission ID = o4auuo 

date = 2021-06-20 18:41:27 

Is it too late to buy $AMC ?
Submission ID = o4tyq4 

date = 2021-06-21 13:28:29 

Mitigating Risks and Maximizing Returns using Collateral Plays - In Depth Trade T

Record Stock Sales From Money-Losing Firms Ring the Alarm Bells
Submission ID = o97np8 

date = 2021-06-27 23:57:12 

Daily option statistics for WISH, AMC, BB, GME, CLNE, CLOV, WKHS, CLF, TSLA, and others.
Submission ID = o542tb 

date = 2021-06-21 21:02:45 

$AMC Covered Options Trading
Submission ID = o5tjsh 

date = 2021-06-22 19:18:04 

Is TD broken or do I actually owe $1,000,000?
Submission ID = o8qy98 

date = 2021-06-27 06:35:49 

Daily Popular Tickers Thread for June 21, 2021 - AMC | SPY | BB
Submission ID = o4rqwu 

date = 2021-06-21 11:00:39 

Identifying bots on WSB
Submission ID = o7t2nt 

date = 2021-06-25 19:05:18 

$AMC Reversed nicely after the New Moon for the perfect setup for this week's Super Full MOON - If History Repeats Itself, Big Move Incoming!
Submission ID = o4auuo 

date = 2021-06-20 18:41:27 

Is it too late to buy $AMC ?
Submission ID = o4tyq4 

date = 2021-06-21 13:28:29 

Mitigating Risks and Maximizing Returns using Collateral Plays - In Depth Trade T

Record Stock Sales From Money-Losing Firms Ring the Alarm Bells
Submission ID = o97np8 

date = 2021-06-27 23:57:12 

Daily option statistics for WISH, AMC, BB, GME, CLNE, CLOV, WKHS, CLF, TSLA, and others.
Submission ID = o542tb 

date = 2021-06-21 21:02:45 

$AMC Covered Options Trading
Submission ID = o5tjsh 

date = 2021-06-22 19:18:04 

Is TD broken or do I actually owe $1,000,000?
Submission ID = o8qy98 

date = 2021-06-27 06:35:49 

Daily Popular Tickers Thread for June 21, 2021 - AMC | SPY | BB
Submission ID = o4rqwu 

date = 2021-06-21 11:00:39 

Identifying bots on WSB
Submission ID = o7t2nt 

date = 2021-06-25 19:05:18 

$AMC Reversed nicely after the New Moon for the perfect setup for this week's Super Full MOON - If History Repeats Itself, Big Move Incoming!
Submission ID = o4auuo 

date = 2021-06-20 18:41:27 

Is it too late to buy $AMC ?
Submission ID = o4tyq4 

date = 2021-06-21 13:28:29 

Mitigating Risks and Maximizing Returns using Collateral Plays - In Depth Trade T

Record Stock Sales From Money-Losing Firms Ring the Alarm Bells
Submission ID = o97np8 

date = 2021-06-27 23:57:12 

Daily option statistics for WISH, AMC, BB, GME, CLNE, CLOV, WKHS, CLF, TSLA, and others.
Submission ID = o542tb 

date = 2021-06-21 21:02:45 

$AMC Covered Options Trading
Submission ID = o5tjsh 

date = 2021-06-22 19:18:04 

Is TD broken or do I actually owe $1,000,000?
Submission ID = o8qy98 

date = 2021-06-27 06:35:49 

Daily Popular Tickers Thread for June 21, 2021 - AMC | SPY | BB
Submission ID = o4rqwu 

date = 2021-06-21 11:00:39 

Identifying bots on WSB
Submission ID = o7t2nt 

date = 2021-06-25 19:05:18 

$AMC Reversed nicely after the New Moon for the perfect setup for this week's Super Full MOON - If History Repeats Itself, Big Move Incoming!
Submission ID = o4auuo 

date = 2021-06-20 18:41:27 

Is it too late to buy $AMC ?
Submission ID = o4tyq4 

date = 2021-06-21 13:28:29 

Mitigating Risks and Maximizing Returns using Collateral Plays - In Depth Trade T

...

Record Stock Sales From Money-Losing Firms Ring the Alarm Bells
Submission ID = o97np8 

date = 2021-06-27 23:57:12 

Daily option statistics for WISH, AMC, BB, GME, CLNE, CLOV, WKHS, CLF, TSLA, and others.
Submission ID = o542tb 

date = 2021-06-21 21:02:45 

$AMC Covered Options Trading
Submission ID = o5tjsh 

date = 2021-06-22 19:18:04 

Is TD broken or do I actually owe $1,000,000?
Submission ID = o8qy98 

date = 2021-06-27 06:35:49 

Daily Popular Tickers Thread for June 21, 2021 - AMC | SPY | BB
Submission ID = o4rqwu 

date = 2021-06-21 11:00:39 

Identifying bots on WSB
Submission ID = o7t2nt 

date = 2021-06-25 19:05:18 

$AMC Reversed nicely after the New Moon for the perfect setup for this week's Super Full MOON - If History Repeats Itself, Big Move Incoming!
Submission ID = o4auuo 

date = 2021-06-20 18:41:27 

Is it too late to buy $AMC ?
Submission ID = o4tyq4 

date = 2021-06-21 13:28:29 

Mitigating Risks and Maximizing Returns using Collateral Plays - In Depth Trade T

Record Stock Sales From Money-Losing Firms Ring the Alarm Bells
Submission ID = o97np8 

date = 2021-06-27 23:57:12 

Daily option statistics for WISH, AMC, BB, GME, CLNE, CLOV, WKHS, CLF, TSLA, and others.
Submission ID = o542tb 

date = 2021-06-21 21:02:45 

$AMC Covered Options Trading
Submission ID = o5tjsh 

date = 2021-06-22 19:18:04 

Is TD broken or do I actually owe $1,000,000?
Submission ID = o8qy98 

date = 2021-06-27 06:35:49 

Daily Popular Tickers Thread for June 21, 2021 - AMC | SPY | BB
Submission ID = o4rqwu 

date = 2021-06-21 11:00:39 

Identifying bots on WSB
Submission ID = o7t2nt 

date = 2021-06-25 19:05:18 

$AMC Reversed nicely after the New Moon for the perfect setup for this week's Super Full MOON - If History Repeats Itself, Big Move Incoming!
Submission ID = o4auuo 

date = 2021-06-20 18:41:27 

Is it too late to buy $AMC ?
Submission ID = o4tyq4 

date = 2021-06-21 13:28:29 

Mitigating Risks and Maximizing Returns using Collateral Plays - In Depth Trade T

Record Stock Sales From Money-Losing Firms Ring the Alarm Bells
Submission ID = o97np8 

date = 2021-06-27 23:57:12 

Daily option statistics for WISH, AMC, BB, GME, CLNE, CLOV, WKHS, CLF, TSLA, and others.
Submission ID = o542tb 

date = 2021-06-21 21:02:45 

$AMC Covered Options Trading
Submission ID = o5tjsh 

date = 2021-06-22 19:18:04 

Is TD broken or do I actually owe $1,000,000?
Submission ID = o8qy98 

date = 2021-06-27 06:35:49 

Daily Popular Tickers Thread for June 21, 2021 - AMC | SPY | BB
Submission ID = o4rqwu 

date = 2021-06-21 11:00:39 

Identifying bots on WSB
Submission ID = o7t2nt 

date = 2021-06-25 19:05:18 

$AMC Reversed nicely after the New Moon for the perfect setup for this week's Super Full MOON - If History Repeats Itself, Big Move Incoming!
Submission ID = o4auuo 

date = 2021-06-20 18:41:27 

Is it too late to buy $AMC ?
Submission ID = o4tyq4 

date = 2021-06-21 13:28:29 

Mitigating Risks and Maximizing Returns using Collateral Plays - In Depth Trade T

Record Stock Sales From Money-Losing Firms Ring the Alarm Bells
Submission ID = o97np8 

date = 2021-06-27 23:57:12 

Daily option statistics for WISH, AMC, BB, GME, CLNE, CLOV, WKHS, CLF, TSLA, and others.
Submission ID = o542tb 

date = 2021-06-21 21:02:45 

$AMC Covered Options Trading
Submission ID = o5tjsh 

date = 2021-06-22 19:18:04 

Is TD broken or do I actually owe $1,000,000?
Submission ID = o8qy98 

date = 2021-06-27 06:35:49 

Daily Popular Tickers Thread for June 21, 2021 - AMC | SPY | BB
Submission ID = o4rqwu 

date = 2021-06-21 11:00:39 

Identifying bots on WSB
Submission ID = o7t2nt 

date = 2021-06-25 19:05:18 

$AMC Reversed nicely after the New Moon for the perfect setup for this week's Super Full MOON - If History Repeats Itself, Big Move Incoming!
Submission ID = o4auuo 

date = 2021-06-20 18:41:27 

Is it too late to buy $AMC ?
Submission ID = o4tyq4 

date = 2021-06-21 13:28:29 

Mitigating Risks and Maximizing Returns using Collateral Plays - In Depth Trade T

Record Stock Sales From Money-Losing Firms Ring the Alarm Bells
Submission ID = o97np8 

date = 2021-06-27 23:57:12 

Daily option statistics for WISH, AMC, BB, GME, CLNE, CLOV, WKHS, CLF, TSLA, and others.
Submission ID = o542tb 

date = 2021-06-21 21:02:45 

$AMC Covered Options Trading
Submission ID = o5tjsh 

date = 2021-06-22 19:18:04 

Is TD broken or do I actually owe $1,000,000?
Submission ID = o8qy98 

date = 2021-06-27 06:35:49 

Daily Popular Tickers Thread for June 21, 2021 - AMC | SPY | BB
Submission ID = o4rqwu 

date = 2021-06-21 11:00:39 

Identifying bots on WSB
Submission ID = o7t2nt 

date = 2021-06-25 19:05:18 

$AMC Reversed nicely after the New Moon for the perfect setup for this week's Super Full MOON - If History Repeats Itself, Big Move Incoming!
Submission ID = o4auuo 

date = 2021-06-20 18:41:27 

Is it too late to buy $AMC ?
Submission ID = o4tyq4 

date = 2021-06-21 13:28:29 

Mitigating Risks and Maximizing Returns using Collateral Plays - In Depth Trade T

In [22]:
# define a submission object
Post1 = reddit.submission(id='ok16jr')

In [23]:
# store all comments scraped from submission object in a list 
Comments_All = []
Post1.comments.replace_more(limit=None)
for comments in Post1.comments.list():
    Comments_All.append(comments.body)

print(Comments_All, '\n')
print('Total Comments Scraped = ', (len(Comments_All)))


['Bought TLRY at 17 and CLOV at 18.05. This thread has made me feel better. Thanks foolios', 'Where tickets come to die', "I bought an AMC 8/20 put.  It should go up now.  You're welcome!", 'Where’s my bb thread?', "I regret jumping on the BB and CLOV bandwagon. I'm down $2.5K combined on both but thankfully they are shares. BB @ 16 and CLOV @ 14.8. Thinking of buying more CLOV to average down to $11.2 but I'm hesitating on putting more money in this lol.", 'CLOV bag holder squad checking in.', 'Yawn. Someone wake me up when CLOV starts moving in the right direction for a decent amount of time', 'BB', 'Ima need CLOV to do that healthcare thing those 4 am guys talked about', 'Maybe I should buy some puts on AMC.  Then it will moooooon', 'That volume on AMC, people seem to like the discount', "I mean if the whole market is tanking I guess I can't be too upset (but I am upset)", 'If CLOV hits $30 I’ll make $40k. If it hits below $9 I’ll have 0. Oh and all this by Friday. Halp', 'Oh shit, 

### Preprocess The Comments

In [24]:
# convert to a string object
List1 = Comments_All
List1 = [str(i) for i in List1]
string_raw = ' , '.join(List1) 
string_raw

'Bought TLRY at 17 and CLOV at 18.05. This thread has made me feel better. Thanks foolios , Where tickets come to die , I bought an AMC 8/20 put.  It should go up now.  You\'re welcome! , Where’s my bb thread? , I regret jumping on the BB and CLOV bandwagon. I\'m down $2.5K combined on both but thankfully they are shares. BB @ 16 and CLOV @ 14.8. Thinking of buying more CLOV to average down to $11.2 but I\'m hesitating on putting more money in this lol. , CLOV bag holder squad checking in. , Yawn. Someone wake me up when CLOV starts moving in the right direction for a decent amount of time , BB , Ima need CLOV to do that healthcare thing those 4 am guys talked about , Maybe I should buy some puts on AMC.  Then it will moooooon , That volume on AMC, people seem to like the discount , I mean if the whole market is tanking I guess I can\'t be too upset (but I am upset) , If CLOV hits $30 I’ll make $40k. If it hits below $9 I’ll have 0. Oh and all this by Friday. Halp , Oh shit, NOK is a m

In [27]:
# remove emojis
string_noemoji = emoji.get_emoji_regexp().sub(u'', string_raw)
string_noemoji

'Bought TLRY at 17 and CLOV at 18.05. This thread has made me feel better. Thanks foolios , Where tickets come to die , I bought an AMC 8/20 put.  It should go up now.  You\'re welcome! , Where’s my bb thread? , I regret jumping on the BB and CLOV bandwagon. I\'m down $2.5K combined on both but thankfully they are shares. BB @ 16 and CLOV @ 14.8. Thinking of buying more CLOV to average down to $11.2 but I\'m hesitating on putting more money in this lol. , CLOV bag holder squad checking in. , Yawn. Someone wake me up when CLOV starts moving in the right direction for a decent amount of time , BB , Ima need CLOV to do that healthcare thing those 4 am guys talked about , Maybe I should buy some puts on AMC.  Then it will moooooon , That volume on AMC, people seem to like the discount , I mean if the whole market is tanking I guess I can\'t be too upset (but I am upset) , If CLOV hits $30 I’ll make $40k. If it hits below $9 I’ll have 0. Oh and all this by Friday. Halp , Oh shit, NOK is a m

In [28]:
# tokenize and clean strings
tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|http\S+')
tokenized_string = tokenizer.tokenize(string_noemoji)
tokenized_string

['Bought',
 'TLRY',
 'at',
 '17',
 'and',
 'CLOV',
 'at',
 '18',
 '05',
 'This',
 'thread',
 'has',
 'made',
 'me',
 'feel',
 'better',
 'Thanks',
 'foolios',
 'Where',
 'tickets',
 'come',
 'to',
 'die',
 'I',
 'bought',
 'an',
 'AMC',
 '8',
 '20',
 'put',
 'It',
 'should',
 'go',
 'up',
 'now',
 'You',
 're',
 'welcome',
 'Where',
 's',
 'my',
 'bb',
 'thread',
 'I',
 'regret',
 'jumping',
 'on',
 'the',
 'BB',
 'and',
 'CLOV',
 'bandwagon',
 'I',
 'm',
 'down',
 '$2.5',
 'K',
 'combined',
 'on',
 'both',
 'but',
 'thankfully',
 'they',
 'are',
 'shares',
 'BB',
 '16',
 'and',
 'CLOV',
 '14',
 '8',
 'Thinking',
 'of',
 'buying',
 'more',
 'CLOV',
 'to',
 'average',
 'down',
 'to',
 '$11.2',
 'but',
 'I',
 'm',
 'hesitating',
 'on',
 'putting',
 'more',
 'money',
 'in',
 'this',
 'lol',
 'CLOV',
 'bag',
 'holder',
 'squad',
 'checking',
 'in',
 'Yawn',
 'Someone',
 'wake',
 'me',
 'up',
 'when',
 'CLOV',
 'starts',
 'moving',
 'in',
 'the',
 'right',
 'direction',
 'for',
 'a',
 'dece

In [29]:
# convert Tokens into lowercase letters
lc_tokenized_string = [word.lower() for word in tokenized_string]
lc_tokenized_string

['bought',
 'tlry',
 'at',
 '17',
 'and',
 'clov',
 'at',
 '18',
 '05',
 'this',
 'thread',
 'has',
 'made',
 'me',
 'feel',
 'better',
 'thanks',
 'foolios',
 'where',
 'tickets',
 'come',
 'to',
 'die',
 'i',
 'bought',
 'an',
 'amc',
 '8',
 '20',
 'put',
 'it',
 'should',
 'go',
 'up',
 'now',
 'you',
 're',
 'welcome',
 'where',
 's',
 'my',
 'bb',
 'thread',
 'i',
 'regret',
 'jumping',
 'on',
 'the',
 'bb',
 'and',
 'clov',
 'bandwagon',
 'i',
 'm',
 'down',
 '$2.5',
 'k',
 'combined',
 'on',
 'both',
 'but',
 'thankfully',
 'they',
 'are',
 'shares',
 'bb',
 '16',
 'and',
 'clov',
 '14',
 '8',
 'thinking',
 'of',
 'buying',
 'more',
 'clov',
 'to',
 'average',
 'down',
 'to',
 '$11.2',
 'but',
 'i',
 'm',
 'hesitating',
 'on',
 'putting',
 'more',
 'money',
 'in',
 'this',
 'lol',
 'clov',
 'bag',
 'holder',
 'squad',
 'checking',
 'in',
 'yawn',
 'someone',
 'wake',
 'me',
 'up',
 'when',
 'clov',
 'starts',
 'moving',
 'in',
 'the',
 'right',
 'direction',
 'for',
 'a',
 'dece

In [40]:
# remove stopwords *** stopwords are words that do not add much information to a sentence
spacy_nlp = en_core_web_sm.load()
all_sw = spacy_nlp.Defaults.stop_words

text = lc_tokenized_string
tokens_wosm = [word for word in text if not word in all_sw]

tokens_wosm



['bought',
 'tlry',
 '17',
 'clov',
 '18',
 '05',
 'thread',
 'feel',
 'better',
 'thanks',
 'foolios',
 'tickets',
 'come',
 'die',
 'bought',
 'amc',
 '8',
 '20',
 'welcome',
 's',
 'bb',
 'thread',
 'regret',
 'jumping',
 'bb',
 'clov',
 'bandwagon',
 'm',
 '$2.5',
 'k',
 'combined',
 'thankfully',
 'shares',
 'bb',
 '16',
 'clov',
 '14',
 '8',
 'thinking',
 'buying',
 'clov',
 'average',
 '$11.2',
 'm',
 'hesitating',
 'putting',
 'money',
 'lol',
 'clov',
 'bag',
 'holder',
 'squad',
 'checking',
 'yawn',
 'wake',
 'clov',
 'starts',
 'moving',
 'right',
 'direction',
 'decent',
 'time',
 'bb',
 'ima',
 'need',
 'clov',
 'healthcare',
 'thing',
 '4',
 'guys',
 'talked',
 'maybe',
 'buy',
 'puts',
 'amc',
 'moooooon',
 'volume',
 'amc',
 'people',
 'like',
 'discount',
 'mean',
 'market',
 'tanking',
 'guess',
 't',
 'upset',
 'upset',
 'clov',
 'hits',
 '$30',
 'll',
 '$40',
 'k',
 'hits',
 '$9',
 'll',
 '0',
 'oh',
 'friday',
 'halp',
 'oh',
 'shit',
 'nok',
 'meme',
 'stock',
 '

In [41]:
cleaned_comments = tokens_wosm

### Pre-Processing vs. Post-Processing

In [43]:
print("Original length of words = ", (len(string_raw)))
print("Number of words after removing emojis = ", (len(string_noemoji)))
print("Number of words after removing tokenizing and cleaning = ", (len(tokenized_string)))
print("Number of words after removing stop words = ", (len(tokens_wosm)))

Original length of words =  55981
Number of words after removing emojis =  55685
Number of words after removing tokenizing and cleaning =  10680
Number of words after removing stop words =  5431


In [48]:
import pickle
with open('cleaned_comments.ob', 'wb') as fp:
    pickle.dump(cleaned_comments, fp) 